In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode, lower, regexp_extract

In [2]:
spark = (SparkSession
         .builder
         .config("spark.sql.repl.eagerEval.enabled", "True")
         .getOrCreate())

In [3]:
book = spark.read.text("../../data/gutenberg_books/1342-0.txt")
type(book)

pyspark.sql.dataframe.DataFrame

In [4]:
lines = book.select(split(book["value"], " ").alias("line"))
lines

line
"[The, Project, Gu..."
[]
"[This, eBook, is,..."
"[almost, no, rest..."
"[re-use, it, unde..."
"[with, this, eBoo..."
[]
[]
"[Title:, Pride, a..."
[]


In [11]:
words = lines.select(explode(book["line"]).alias("word"))
words

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `line` cannot be resolved. Did you mean one of the following? [`value`].

In [ ]:
words_lower = words.select(lower(col("word")).alias("word_lower"))
words_lower

word_lower
the
project
gutenberg
ebook
of
pride
and
"prejudice,"
by
jane


In [ ]:
words_clean = words_lower.select(
    regexp_extract(col("word_lower"), "[a-z]*", 0).alias("word")
)
words_clean.toPandas()

,word
0,the
1,project
2,gutenberg
3,ebook
4,of
...,...
127363,to
127364,hear
127365,about
127366,new


In [ ]:
words_nonull = words_clean.where(col("word") != "")
words_nonull.toPandas()

,word
0,the
1,project
2,gutenberg
3,ebook
4,of
...,...
122152,to
122153,hear
122154,about
122155,new
